In [7]:
!pip install langgraph langchain-core gigachat pdfplumber PyPDF2 python-dotenv

  Obtaining dependency information for PyPDF2 from https://files.pythonhosted.org/packages/8e/5e/c86a5643653825d3c913719e788e41386bee415c2b87b4f955432f2de6b2/pypdf2-3.0.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/232.6 kB ? eta -:--:--
   ------- -------------------------------- 41.0/232.6 kB 2.0 MB/s eta 0:00:01
   -------------------------------------- - 225.3/232.6 kB 3.5 MB/s eta 0:00:01
   ---------------------------------------- 232.6/232.6 kB 2.9 MB/s eta 0:00:00


In [1]:
import os
print(os.getcwd())
print(os.listdir())


C:\Users\aleks\DS projects\pd_agent_system
['.env', '.ipynb_checkpoints', 'agent_solution.ipynb']


In [1]:
import os
from typing import Dict, Any, List, Optional
from dataclasses import dataclass
import pdfplumber
import json
from dotenv import load_dotenv

from langgraph.graph import StateGraph, END
from gigachat import GigaChat

load_dotenv()

# Загрузка переменных окружения
GIGACHAT_API_KEY = os.getenv("GIGACHAT_API_KEY")


In [2]:

#Состояние графа
@dataclass
class AgentState:
    messages: List[Dict[str, Any]]
    current_agent: str
    user_query: str
    extracted_text: Optional[str] = None
    final_response: Optional[str] = None


#  Базовый агент
class BaseAgent:
    def __init__(self, name: str, tools: List[Any]):
        self.name = name
        self.tools = tools
        self.model = GigaChat(credentials=GIGACHAT_API_KEY, verify_ssl_certs=False)

    def chat(self, user_query: str, history: Optional[List[Dict[str, str]]] = None) -> str:
        """
        Обращение к LLM с учётом истории диалога.
        history: [{"role": "user"/"assistant", "content": "..."}]
        """
        if history:
            hist_text = ""
            # Берём только последние N сообщений, чтобы не раздуть контекст
            for msg in history[-10:]:
                role = "Пользователь" if msg["role"] == "user" else "Ассистент"
                hist_text += f"{role}: {msg['content']}\n"

            prompt = (
                f"{hist_text}\n"
                f"Пользователь: {user_query}\n"
                f"Ассистент:"
            )
        else:
            prompt = user_query

        resp = self.model.chat(prompt)
        return resp.choices[0].message.content
    

#  Оркестратор
class OrchestratorAgent(BaseAgent):
    def __init__(self):
        super().__init__("orchestrator", [])

    def route_query(self, state: AgentState) -> str:
        """Определяет, какой агент должен обработать запрос"""
        query = state.user_query.lower()

        career_keywords = ['резюме', 'ваканси', 'собеседован', 'карьер', 'работ', 'cv']
        learning_keywords = ['обучен', 'изуч', 'курс', 'материал', 'roadmap', 'концепц']

        career_score = sum(1 for keyword in career_keywords if keyword in query)
        learning_score = sum(1 for keyword in learning_keywords if keyword in query)

        if career_score > learning_score:
            return "career_agent"
        elif learning_score > career_score:
            return "learning_agent"
        else:
            # если примерно одинаково — по умолчанию считаем, что это про обучение
            return "learning_agent"

In [3]:
# Агент карьеры
class CareerAgent(BaseAgent):
    def __init__(self):
        tools = [
            self.analyze_resume_tool,
            self.analyze_vacancy_tool,
            self.generate_interview_questions_tool,
            self.extract_pdf_text_tool,
        ]
        super().__init__("career_agent", tools)

    def extract_pdf_text_tool(self, pdf_path: str) -> str:
        """Извлекает текст из PDF файла"""
        try:
            text = ""
            with pdfplumber.open(pdf_path) as pdf:
                for page in pdf.pages:
                    text += page.extract_text() or ""
            return text[:2000]
        except Exception as e:
            return f"Ошибка при чтении PDF: {str(e)}"

    def analyze_resume_tool(self, pdf_path: str) -> str:
        """Анализирует резюме из PDF файла"""
        text = self.extract_pdf_text_tool(pdf_path)

        prompt = f"""
        Проанализируй это резюме и дай рекомендации по улучшению:

        {text}

        Критерии анализа:
        1. Структура и читаемость
        2. Наличие конкретных достижений и метрик
        3. Соответствие современным требованиям
        4. Оптимизация под ATS системы
        5. Профессиональное впечатление

        Верни структурированный анализ.
        """

        response = self.model.chat(prompt)
        return response.choices[0].message.content

    def analyze_vacancy_tool(self, pdf_path: str) -> str:
        """Анализирует вакансию из PDF файла"""
        text = self.extract_pdf_text_tool(pdf_path)

        prompt = f"""
        Проанализируй вакансию и определи "красные флаги":

        {text}

        Ищи:
        1. Размытые обязанности
        2. Непрозрачные условия оплаты
        3. Завышенные требования
        4. Признаки токсичной рабочей среды
        5. Противоречивые условия

        Верни анализ с выделением позитивных и негативных аспектов.
        """

        response = self.model.chat(prompt)
        return response.choices[0].message.content

    def generate_interview_questions_tool(self, position: str, level: str = "middle") -> str:
        """Генерирует вопросы для собеседования"""
        prompt = f"""
        Сгенерируй список из 10-15 вопросов для собеседования на позицию {position} уровня {level}.
        Включи:
        - Технические вопросы
        - Поведенческие вопросы
        - Вопросы о мотивации
        - Кейсовые задания

        Для каждого вопроса укажи, на что обращать внимание в ответе.
        """

        response = self.model.chat(prompt)
        return response.choices[0].message.content

In [ ]:
# Агент обучения
class LearningAgent(BaseAgent):
    def __init__(self):
        tools = [
            self.create_roadmap_tool,
            self.recommend_materials_tool,
            self.explain_concept_tool,
            self.extract_pdf_text_tool,
        ]
        super().__init__("learning_agent", tools)

    def extract_pdf_text_tool(self, pdf_path: str) -> str:
        """Извлекает текст из PDF файла"""
        try:
            text = ""
            with pdfplumber.open(pdf_path) as pdf:
                for page in pdf.pages:
                    text += page.extract_text() or ""
            return text[:2000]
        except Exception as e:
            return f"Ошибка при чтении PDF: {str(e)}"

    def create_roadmap_tool(self, profession: str,) -> str:
        """Создает образовательный roadmap"""
        prompt = f"""
        Изучи запрос пользователя и определи из его запроса на какой срок будет создаваться учебный план.
        Создай детальный образовательный roadmap для профессии {profession} на период, который ты определил из запроса ранее

        Структура:
        1. Цели обучения
        2. Поэтапный план (с временными метками)
        3. Ключевые навыки для освоения
        4. Рекомендуемые проекты для практики
        5. Метрики успеха
        6. Чекпоинты для самопроверки

        Сделай roadmap практичным и достижимым.
        """

        response = self.model.chat(prompt)
        return response.choices[0].message.content

    def recommend_materials_tool(self, topic: str, level: str = "beginner") -> str:
        """Рекомендует обучающие материалы"""
        prompt = f"""
        Подбери рекомендации обучающих материалов по теме {topic} для уровня {level}.

        Включи:
        - Книги (с обоснованием выбора)
        - Онлайн курсы
        - Статьи и блоги
        - Видео материалы
        - Практические задания
        - Сообщества для обучения

        Укажи примерное время на освоение каждого ресурса.
        """

        response = self.model.chat(prompt)
        return response.choices[0].message.content

    def explain_concept_tool(self, concept: str, explanation_level: str = "новичок") -> str:
        """Объясняет сложные концепции простым языком"""
        prompt = f"""
        Объясни концепцию {concept} для уровня {explanation_level}.

        Используй:
        - Простые аналогии
        - Конкретные примеры
        - Визуальные метафоры
        - Практические применения

        Избегай сложной терминологии, объясняй как для новичка.
        """

        response = self.model.chat(prompt)
        return response.choices[0].message.content

In [ ]:
# Мультиагентная система
class MultiAgentSystem:
    def __init__(self):
        self.orchestrator = OrchestratorAgent()
        self.career_agent = CareerAgent()
        self.learning_agent = LearningAgent()
        self.graph = self._build_graph()
        self.chat_history: List[Dict[str, str]] = []
    def _append_history(self, user_query: str, response: str) -> None:
        """Сохраняем последний обмен в истории."""
        self.chat_history.append({"role": "user", "content": user_query})
        self.chat_history.append({"role": "assistant", "content": response})


    # Построение графа
    def _build_graph(self) -> StateGraph:
        workflow = StateGraph(AgentState)

        workflow.add_node("orchestrator", self._orchestrator_node)
        workflow.add_node("career_agent", self._career_agent_node)
        workflow.add_node("learning_agent", self._learning_agent_node)

        workflow.set_entry_point("orchestrator")

        workflow.add_conditional_edges(
            "orchestrator",
            self._route_after_orchestrator,
            {
                "career_agent": "career_agent",
                "learning_agent": "learning_agent",
            },
        )

        workflow.add_edge("career_agent", END)
        workflow.add_edge("learning_agent", END)

        return workflow.compile()

    # Узлы графа
    def _orchestrator_node(self, state: AgentState) -> AgentState:
        agent_type = self.orchestrator.route_query(state)
        state.current_agent = agent_type
        return state

    def _career_agent_node(self, state: AgentState) -> AgentState:
        response = self._process_with_agent(self.career_agent, state.user_query, self.chat_history)
        state.final_response = response
        return state

    def _learning_agent_node(self, state: AgentState) -> AgentState:
        response = self._process_with_agent(self.learning_agent, state.user_query, self.chat_history)
        state.final_response = response
        return state

In [2]:
#  Логика работы агентов
    def _process_with_agent(self, agent: BaseAgent, query: str, history: Optional[List[Dict[str, str]]] = None) -> str:
        """Обрабатывает запрос с помощью указанного агента (без JSON-режиссуры)"""
        q = query.lower()
        if isinstance(agent, CareerAgent):
            if "собеседован" in q and "вопрос" in q:
                return agent.generate_interview_questions_tool(position=query)

            if "ваканси" in q and ".pdf" not in q:
                prompt = (
                    f"Проанализируй эту вакансию и укажи красные и зелёные флаги.\n\n"
                    f"Текст запроса пользователя:\n{query}"
                )
        
                resp_text = agent.chat(prompt, history=history)
                return resp_text

            # Всё остальное по карьере — просто отдаём LLM с историей
            resp_text = agent.chat(query, history=history)
            return resp_text

        # Агент обучения
        if isinstance(agent, LearningAgent):
            if "roadmap" in q or "дорожн" in q or "план" in q:
                return agent.create_roadmap_tool(profession=query)

            if "объясн" in q or "концепц" in q:
                return agent.explain_concept_tool(concept=query)

            resp_text = agent.chat(query, history=history)
            return resp_text

        # На всякий случай fallback
        else:
            resp_text = agent.chat(query, history=history)
        return resp_text

    def _route_after_orchestrator(self, state: AgentState) -> str:
        return state.current_agent

    # Текст про предназначение 
    def _purpose_message(self) -> str:
        return (
            "Я специализированный ассистент по вопросам карьеры и обучения.\n\n"
            "Моё основное предназначение:\n"
            "• анализировать резюме и вакансии,\n"
            "• помогать с подготовкой к собеседованиям,\n"
            "• строить образовательные roadmap’ы и планы развития,\n"
            "• подбирать материалы для обучения и объяснять сложные концепции.\n\n"
            "Если у тебя есть вопрос про карьеру, обучение или резюме — сформулируй его, "
            "и я помогу максимально детально. По другим темам я, к сожалению, не отвечаю."
        )

    def process_query(self, user_query: str, resume_file_path: Optional[str] = None) -> str:
        q = user_query.lower()

        career_keywords = ['резюме', 'ваканси', 'собеседован', 'карьер', 'работ', 'cv']
        learning_keywords = ['обучен', 'изуч', 'курс', 'материал', 'roadmap', 'концепц', 'учиться', 'обучаться']

        tech_keywords = [
            'программир', 'python', 'java', 'c++', 'c#', 'sql',
            'алгоритм', 'структур данных', 'машинное обучен', 'ml',
            'data', 'аналитик', 'backend', 'frontend', 'devops',
            'langgraph', 'лангграф', 'llm', 'агент', 'multi-agent'
        ]

        base_match = any(k in q for k in career_keywords + learning_keywords)

        is_explain_tech = ("объясн" in q) and any(k in q for k in tech_keywords)

        is_career_or_learning = base_match or is_explain_tech

        if not is_career_or_learning:
            response = self._purpose_message()
            self._append_history(user_query, response)
            return response

        resume_keywords = ['резюме', 'cv']
        analyze_keywords = ['анализ', 'проанализируй', 'оцен', 'разбор']

        is_resume_query = any(k in q for k in resume_keywords)
        is_analyze_request = any(k in q for k in analyze_keywords)

        if is_resume_query and is_analyze_request and resume_file_path is not None:
            response = self.career_agent.analyze_resume_tool(resume_file_path)
            self._append_history(user_query, response)
            return response

        initial_state = AgentState(
            messages=[],
            current_agent="orchestrator",
            user_query=user_query,
        )
        result = self.graph.invoke(initial_state)
        response = result.get("final_response", "Не удалось обработать запрос")

        self._append_history(user_query, response)
        return response



In [3]:
system = MultiAgentSystem()

while True:
    user_text = input("Ты: ")
    if user_text.strip().lower() in {"выход", "exit", "quit"}:
        break

    resume_path = None
    q = user_text.lower()
    if "резюме" in q and "анализ" in q:
        resume_path = input("📂 Введи путь к PDF с резюме (например, C:\\Users\\aleks\\resume.pdf): ").strip()

    answer = system.process_query(user_text, resume_file_path=resume_path)
    print("Бот:", answer)
    print("-" * 80)

Ты: объясни мне что такое .invoke в лангграф
Бот: Представь себе такую ситуацию: ты хочешь сделать вкусный чайный напиток дома — взять заварочный чайник, воду, чайный пакетик и сахар (если хочется сладкого). Ты берёшь кружку, наполняешь её водой, ставишь чайник на плиту, ждёшь закипания воды, добавляешь туда чайный пакетик и варишь пару минут, потом вынимаешь пакетик, даёшь чаю немного остыть и пьёшь, наслаждаясь результатом.

Аналогия с `invoke` в Langgraph примерно такая же простая!

## Что такое `.invoke`?

`.invoke()` — это команда, которую мы даём нашему виртуальному помощнику (или AI), чтобы тот выполнил какое-то действие или работу. Например, **мы говорим нашему помощнику:**  
**«Напиши письмо моему другу»**,  
**«Покажи список моих дел на сегодня»**,  
**«Сгенери картинку по моему запросу»**.  
И наш помощник исполняет наши команды, выполняя нужные нам задания, используя встроенные инструменты и умения.

### Почему использовать `.invoke()` удобно?

- Представим, что тебе нужно 


KeyboardInterrupt

